In [33]:
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from datetime import datetime
from dateutil import parser, tz
import pandas as pd

def run():
    
    def calc_status(lesson, assignment_file,bbcolumn):
        return "N/A"
        # if parser.parse(assignment_date.value)>= parser.parse(datetime.now().isoformat()):
        #     return "Not Due Yet"
        # elif not tt.is_fetched(lesson,assignment_file):
        #     return "Can Be Fetched"
        # elif not tt.bbfile_exists(bbcolumn):
        #     return "Fetched"
        # else:
        #     return "BB File Created"
            
    display(HTML(f"<h2>Teacher Tools:</h2>"))
    df = pd.read_csv("admin/data/tst101-assignments.csv")
    
    lesson_col = df.columns[0]
    assignment_col = df.columns[1]
    points_col = df.columns[2]
    duedate_col = df.columns[3]
    bb_col = df.columns[4]

#     #df['duedatedt'] = df.apply( lambda row: parser.parse(row[duedate_col]), axis=1)
    units = df[lesson_col].unique().tolist()
    
    course_dropdown = widgets.Dropdown(
        options=['tst101-spring2024', 'ist256-spring2024'],
        value='tst101-spring2024',
        description='Course:',
        layout={'width': 'max-content'},
        disabled=False
    )
    
    cmd = widgets.ToggleButtons(
        options=['View', 'Fetch', 'Blackboard'],
        description='Action:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['View Assignment Details And Status of Assignments', 'Fetch Submissions From Students', 'Create Blackboard File From Graded Assignments'],
        icons=['search', 'download', 'graduation-cap'],
        layout={'width': 'max-content'}
    )

    unit_dropdown = widgets.Dropdown(
        options=units,
        value=units[0],
        description='Lesson:',
        layout={'width': 'max-content'},
        disabled=False
    )

    assignment_dropdown = widgets.Dropdown(
        options=df[df[lesson_col] == unit_dropdown.value].sort_values(assignment_col)[assignment_col].unique().tolist(),
        description='Assignment:',
        layout={'width': 'max-content'},
        disabled=False
    )

    assignment_date = widgets.Text(
        value=df[ df[assignment_col] == assignment_dropdown.value][duedate_col].values[0],
        description='Due Date:',
        disabled=True
    )

    assignment_points = widgets.Text(
        value=str(df[ df[assignment_col] == assignment_dropdown.value][points_col].values[0]),
        description='Total Points:',
        disabled=True
    )

    assignment_bbcolumn = widgets.Text(
        value=str(df[ df[assignment_col] == assignment_dropdown.value][bb_col].values[0]),
        description='BB Column:',
        disabled=True
    )

    assignment_status = widgets.Text(
        value=calc_status(unit_dropdown.value,assignment_dropdown.value,assignment_bbcolumn.value),
        description='Status:',
        disabled=True
    )

    template_dropdown = widgets.Dropdown(
        options=["X","Y"], # tt.get_templates(),
        description='Template:',
        layout={'width': 'max-content'},
        disabled=False
    )

    button = widgets.Button(
        description="Select Assignment",        
        icon="check",
        button_style='primary',
        layout={'width': 'max-content'},
        disabled = False)
    
    output = widgets.Output()
            

    #handlers
    def cmd_on_change(*args):
        button.description = cmd.value
        button.icon = cmd.icons[cmd.index]
        if cmd.index == 0:
            button.layout.display = 'none' 
        else:
            button.layout.display = None
        if cmd.index == 1:
            template_dropdown.layout.display = None 
        else:
            template_dropdown.layout.display = 'none'
            
        
    def unit_dropdown_on_change(*args):
        assignment_dropdown.options = df[df[lesson_col] == unit_dropdown.value].sort_values(assignment_col)[assignment_col].unique().tolist()

    def assignment_dropdown_on_change(*args):
        assignment_date.value = df[ df[assignment_col] == assignment_dropdown.value][duedate_col].values[0]
        assignment_points.value=str(df[ df[assignment_col] == assignment_dropdown.value][points_col].values[0])
        assignment_bbcolumn.value=str(df[ df[assignment_col] == assignment_dropdown.value][bb_col].values[0])
        assignment_status.value=calc_status(unit_dropdown.value,assignment_dropdown.value,assignment_bbcolumn.value)


    def on_button_clicked(b):
        with output:
            clear_output()
            if cmd.index ==  1:
                count, folder_count = tt.fetch_assignments(unit_dropdown.value,assignment_dropdown.value,template_dropdown.value)            
                display(HTML(f"Fetched {count} new assignments. {folder_count} assignments total."))
            elif cmd.index == 2:
                count, grades_file = tt.create_bb_file(unit_dropdown.value,assignment_dropdown.value,assignment_bbcolumn.value)
                display(HTML(f"{count} assignments added to grades file: {grades_file}"))
    

    #events
    cmd.observe(cmd_on_change)
    unit_dropdown.observe(unit_dropdown_on_change)
    assignment_dropdown.observe(assignment_dropdown_on_change)
    button.on_click(on_button_clicked)

    #draw
    display(course_dropdown)
    display(cmd,unit_dropdown, assignment_dropdown, assignment_date, assignment_points, assignment_bbcolumn, assignment_status, template_dropdown, button, output)
    cmd_on_change()
    assignment_dropdown_on_change()
    
    



In [34]:
if __name__ == "__main__":
    run()

Dropdown(description='Course:', layout=Layout(width='max-content'), options=('tst101-spring2024', 'ist256-spri…

ToggleButtons(description='Action:', icons=('search', 'download', 'graduation-cap'), layout=Layout(width='max-…

Dropdown(description='Lesson:', layout=Layout(width='max-content'), options=('A',), value='A')

Dropdown(description='Assignment:', layout=Layout(width='max-content'), options=('A1.ipynb', 'A2.ipynb'), valu…

Text(value='2024-01-05 23:59', description='Due Date:', disabled=True)

...

Text(value='342634', description='BB Column:', disabled=True)

Text(value='N/A', description='Status:', disabled=True)

Dropdown(description='Template:', layout=Layout(width='max-content'), options=('X', 'Y'), value='X')

Button(button_style='primary', description='Select Assignment', icon='check', layout=Layout(width='max-content…

Output()